In [1]:
import os

from dotenv import load_dotenv
from langchain import hub
from langchain.agents import AgentExecutor, AgentType, initialize_agent, load_tools
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import (
    JSONAgentOutputParser,
    ReActSingleInputOutputParser,
)
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.tools import ArxivQueryRun, WikipediaQueryRun, tool
from langchain.tools.render import render_text_description_and_args, format_tool_to_openai_function
from langchain.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain.prompts import MessagesPlaceholder
from langchain.schema import ChatMessage, SystemMessage

from llamp.mp.agents import (
    MPSummaryExpert,
    MPThermoExpert,
    MPElasticityExpert,
    MPDielectricExpert,
    MPPiezoelectricExpert,
    MPMagnetismExpert,
    MPElectronicExpert,
    MPSynthesisExpert,
    MPStructureRetriever
)
from llamp.arxiv.agents import ArxivAgent

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)

# OPENAI_GPT_MODEL = "gpt-4-1106-preview"
# OPENAI_GPT_MODEL = "gpt-3.5-turbo-1106"
OPENAI_GPT_MODEL = "gpt-4"

In [2]:
import re

mp_llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo-1106",
    openai_api_key=OPENAI_API_KEY,
    openai_organization=None,
    max_retries=5,
    # streaming=True
)

llm = ChatOpenAI(
    temperature=0.7,
    model="gpt-4",
    openai_api_key=OPENAI_API_KEY,
    openai_organization=None,
    # streaming=True
)

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper())

tools = [
    MPThermoExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPElasticityExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPDielectricExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPMagnetismExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPElectronicExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPPiezoelectricExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPSummaryExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPSynthesisExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPStructureRetriever(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    # ArxivAgent(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    arxiv,
    wikipedia,
]
tools += load_tools(["llm-math"], llm=mp_llm)

prompt = hub.pull("hwchase17/react-multi-input-json")
prompt.messages[0].prompt.template = re.sub(
    r"\s+", " ",
    """You are a data-aware agent that can consult materials-related
    data through Materials Project (MP) database, arXiv, and Wikipedia. Ask 
    user to clarify their queries if needed. Please note that you don't have 
    direct control over MP but through multiple assistant agents to help you. 
    You need to provide complete context in the input for them to do their job.
    """).replace("\n", " ") + prompt.messages[0].prompt.template

prompt = prompt.partial(
    tools=render_text_description_and_args(tools),
    tool_names=", ".join([t.name for t in tools]),
)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | llm.bind(stop=["Observation"])
    # | map_reduce_chain  # TODO: Add map-reduce after LLM
    | JSONAgentOutputParser()
)


conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

agent_kwargs = {
    "handle_parsing_errors": True,
    "extra_prompt_messages": [
        MessagesPlaceholder(variable_name="chat_history"),
        # SystemMessage(content=re.sub(
        #     r"\s+", " ",
        #     """You are a helpful data-aware agent that can consult materials-related
        #     data through Materials Project (MP) database, arXiv, and Wikipedia. Ask 
        #     user to clarify their queries if needed. Please note that you don't have 
        #     direct control to MP but through multiple assistant agents to help you. 
        #     You need to provide complete context for them to do their job. 
        #     """).replace("\n", " ")
        # )
        ],
    "early_stopping_method": 'generate',
}

agent_executor = initialize_agent(
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=20,
    memory=conversational_memory,
    agent_kwargs=agent_kwargs,
    handle_parsing_errors=True,
)

/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(
/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new a

In [3]:
agent_executor.invoke({
    "input": "What are the bulk moduli of the following metals: Sc, Ti, V, Cr, Mn, Fe, Co, Ni, Cu, Zn?",
})



> Entering new AgentExecutor chain...
The user is asking for the bulk modulus of several metals. The bulk modulus is a measure of a substance's resistance to uniform compression. This is a question that can be answered using the MPElasticityExpert tool. I'll ask for the bulk modulus of each metal in turn. Let's start with Scandium (Sc). 
Action:
```
{
  "action": "MPElasticityExpert",
  "action_input": "What is the bulk modulus of Scandium?"
}
```

> Entering new AgentExecutor chain...


/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(
/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


```
{
  "action": "search_materials_elasticity__get",
  "action_input": {
    "formula": "Sc"
  }
}
```{"formula": "Sc"}


Retrieving SummaryDoc documents:   0%|          | 0/10 [00:00<?, ?it/s]

Retrieving ElasticityDoc documents:   0%|          | 0/5 [00:00<?, ?it/s]

...

Action:
```
{
  "action": "search_materials_elasticity__get",
  "action_input": {
    "formula": "Zn",
    "fields": "material_id,formula_pretty,bulk_modulus"
  }
}
```{"formula": "Zn", "fields": "material_id,formula_pretty,bulk_modulus"}


Retrieving SummaryDoc documents:   0%|          | 0/8 [00:00<?, ?it/s]

Retrieving ElasticityDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

[{'formula_pretty': 'Zn', 'material_id': 'mp-79', 'elastic_tensor': {'raw': [[149.94394707539584, 28.63908410291827, 74.98189375739773, -4.7073456244106625e-14, -4.751754545395669e-14, -3.5527136788005e-14], [28.63908410291827, 150.78801544180305, 75.6349454512253, -4.5741188614556437e-14, -4.840572387365681e-14, -2.3337757119295665e-14], [74.98189375739773, 75.6349454512253, 27.30090693351088, 7.549516567451067e-14, 7.638334409421078e-14, 5.062616992290715e-14], [-4.7073456244106625e-14, -4.5741188614556437e-14, 7.549516567451067e-14, 50.97943700003001, 0.0, 7.600146201115776e-30], [-4.751754545395669e-14, -4.840572387365681e-14, 7.638334409421078e-14, 0.0, 51.01396247156045, -1.6701299554935642e-15], [-3.5527136788005e-14, -2.3337757119295665e-14, 5.062616992290715e-14, 7.600146201115776e-30, -1.6701299554935642e-15, 69.6545337313242]], 'ieee_format': [[155.0, 24.0, 75.0, 0.0, -0.0, 0.0], [24.0, 155.0, 75.0, 0.0, 0.0, 0.0], [75.0, 75.0, 27.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 51.0, 0.0

{'input': 'What are the bulk moduli of the following metals: Sc, Ti, V, Cr, Mn, Fe, Co, Ni, Cu, Zn?',
 'chat_history': [],
 'output': 'Here are the bulk moduli of the requested metals: \n\n- Scandium (Sc): Data not available \n- Titanium (Ti): Varies across materials (93 GPa - 198 GPa) \n- Vanadium (V): Approximately 271 GPa \n- Chromium (Cr): Varies depending on the specific material (5.0 GPa - 480.0 GPa) \n- Manganese (Mn): Approximately 96-97 GPa \n- Iron (Fe): Varies depending on the crystal structure (250 GPa - 529 GPa) \n- Cobalt (Co): Approximately 200.587 GPa \n- Nickel (Ni): Approximately 197.345 GPa \n- Copper (Cu): Approximately 151.394 GPa \n- Zinc (Zn): 85.872 GPa'}

In [4]:
print(agent_executor.memory.chat_memory.messages[1].content)

Here are the bulk moduli of the requested metals: 

- Scandium (Sc): Data not available 
- Titanium (Ti): Varies across materials (93 GPa - 198 GPa) 
- Vanadium (V): Approximately 271 GPa 
- Chromium (Cr): Varies depending on the specific material (5.0 GPa - 480.0 GPa) 
- Manganese (Mn): Approximately 96-97 GPa 
- Iron (Fe): Varies depending on the crystal structure (250 GPa - 529 GPa) 
- Cobalt (Co): Approximately 200.587 GPa 
- Nickel (Ni): Approximately 197.345 GPa 
- Copper (Cu): Approximately 151.394 GPa 
- Zinc (Zn): 85.872 GPa


In [5]:

llm_gpt = ChatOpenAI(
    temperature=0.7,
    model="gpt-3.5-turbo-1106",
    openai_api_key=OPENAI_API_KEY,
    openai_organization=None,
    # streaming=True
)

response = llm_gpt.invoke(agent_executor.memory.chat_memory.messages[0].content)


In [6]:
response.content

'The bulk moduli for the metals are as follows (in GPa):\n\n- Sc: 56\n- Ti: 110\n- V: 160\n- Cr: 160\n- Mn: 120\n- Fe: 170\n- Co: 180\n- Ni: 180\n- Cu: 140\n- Zn: 100'

In [7]:
print(response.content)

The bulk moduli for the metals are as follows (in GPa):

- Sc: 56
- Ti: 110
- V: 160
- Cr: 160
- Mn: 120
- Fe: 170
- Co: 180
- Ni: 180
- Cu: 140
- Zn: 100


In [8]:
{
  "sites": [
    {
      "species": [
        {
          "element": "Si",
          "occu": 1
        }
      ],
      "abc": [
        0,
        0,
        0
      ],
      "xyz": [
        0,
        0,
        0
      ],
      "label": "Si",
      "properties": {}
    },
    {
      "species": [
        {
          "element": "Si",
          "occu": 1
        }
      ],
      "abc": [
        0.25,
        0.25,
        0.25
      ],
      "xyz": [
        1.780000e-01,
        1.780000e-01,
        1.780000e-01
      ],
      "label": "Si",
      "properties": {}
    }
  ],
  "lattice": {
    "matrix": [
      [
        3.866002,
        0,
        2.243049
      ],
      [
        3.866002,
        3.866002,
        0
      ],
      [
        0,
        3.866002,
        2.243049
      ]
    ],
    "a": 5.431,
    "b": 5.431,
    "c": 5.431,
    "alpha": 109.471219,
    "beta": 109.471219,
    "gamma": 109.471219,
    "volume": 129.584354898
  },
  "properties": {
    "space_group": {
      "number": 227,
      "source": "spglib",
      "symbol": "Fd-3m"
    }
  }
}

{'sites': [{'species': [{'element': 'Si', 'occu': 1}],
   'abc': [0, 0, 0],
   'xyz': [0, 0, 0],
   'label': 'Si',
   'properties': {}},
  {'species': [{'element': 'Si', 'occu': 1}],
   'abc': [0.25, 0.25, 0.25],
   'xyz': [0.178, 0.178, 0.178],
   'label': 'Si',
   'properties': {}}],
 'lattice': {'matrix': [[3.866002, 0, 2.243049],
   [3.866002, 3.866002, 0],
   [0, 3.866002, 2.243049]],
  'a': 5.431,
  'b': 5.431,
  'c': 5.431,
  'alpha': 109.471219,
  'beta': 109.471219,
  'gamma': 109.471219,
  'volume': 129.584354898},
 'properties': {'space_group': {'number': 227,
   'source': 'spglib',
   'symbol': 'Fd-3m'}}}

In [9]:
from pymatgen.core import Structure

structure = Structure.from_dict(
    {'@module': 'pymatgen.core.structure', '@class': 'Structure', 'charge': 0, 'lattice': {'matrix': [[3.333573, 0.0, 1.924639], [1.111191, 3.142924, 1.924639], [0.0, 0.0, 3.849278]], 'pbc': [True, True, True], 'a': 3.8492784033699095, 'b': 3.8492794116013456, 'c': 3.849278, 'alpha': 60.00001213094421, 'beta': 60.00000346645984, 'gamma': 60.00001097545789, 'volume': 40.32952684741405}, 'properties': {}, 'sites': [{'species': [{'element': 'Si', 'occu': 1}], 'abc': [0.875, 0.875, 0.875], 'xyz': [3.8891685, 2.7500584999999997, 6.7362365], 'properties': {'magmom': -0.0}, 'label': 'Si'}, {'species': [{'element': 'Si', 'occu': 1}], 'abc': [0.125, 0.125, 0.125], 'xyz': [0.5555955, 0.3928655, 0.9623195], 'properties': {'magmom': -0.0}, 'label': 'Si'}, {'species': [{'element': 'Li', 'occu': 1}], 'abc': [0.5, 0.5, 0.5], 'xyz': [2.222382, 1.571462, 1.924639], 'properties': {}, 'label': 'Li'}]}
)

structure.to(filename="../experiments/05-crystal-Si-Li-interstitial.cif", fmt="cif")
structure.to_ase_atoms().write("../experiments/05-crystal-Si-Li-interstitial.xyz")

/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/pymatgen/core/structure.py:990: UserWarning: Not all sites have property magmom. Missing values are set to None.
  warnings.warn(f"Not all sites have property {key}. Missing values are set to None.")


In [10]:
structure = Structure.from_dict(
    {'@module': 'pymatgen.core.structure', '@class': 'Structure', 'charge': 0, 'lattice': {'matrix': [[3.700773, -0.991615, 2.3e-05], [-0.991617, 3.700765, -2.4e-05], [3e-05, -3.2e-05, 6.332622]], 'pbc': [True, True, True], 'a': 3.8313210654137304, 'b': 3.83131385564926, 'c': 6.332622000151911, 'alpha': 90.00070882372823, 'beta': 89.99931892714707, 'gamma': 119.99994034554126, 'volume': 80.50275361604729}, 'properties': {}, 'sites': [{'species': [{'element': 'Si', 'occu': 1}], 'abc': [0.666683, 0.333317, 0.500012], 'xyz': [2.13673464273, 0.572419024076, 3.166394325565], 'properties': {'magmom': 0.0}, 'label': 'Si'}, {'species': [{'element': 'Si', 'occu': 1}], 'abc': [1.5e-05, 0.999985, 9e-06], 'xyz': [-0.99154661388, 3.7006946140120003, 3.2994303e-05], 'properties': {'magmom': 0.0}, 'label': 'Si'}, {'species': [{'element': 'Si', 'occu': 1}], 'abc': [0.666683, 0.333318, 0.873993], 'xyz': [2.136744870543, 0.5724107574489999, 5.534674633723], 'properties': {'magmom': 0.0}, 'label': 'Si'}, {'species': [{'element': 'Si', 'occu': 1}], 'abc': [2e-05, 0.999981, 0.373986], 'xyz': [-0.991512924237, 3.700662885613, 2.368287972208], 'properties': {'magmom': 0.0}, 'label': 'Si'}]}
)

structure.to(filename="../experiments/05-crystal-Si.cif", fmt="cif")
structure.to_ase_atoms().write("../experiments/05-crystal-Si.xyz")

In [11]:

structure = Structure.from_dict(
    {
  "sites": [
    {
      "species": [
        {
          "element": "Si",
          "occu": 1
        }
      ],
      "abc": [0, 0, 0],
      "xyz": [0, 0, 0],
      "label": "Si",
      "properties": {}
    },
    {
      "species": [
        {
          "element": "Si",
          "occu": 1
        }
      ],
      "abc": [0.25, 0.25, 0.25],
      "xyz": [0.178, 0.178, 0.178],
      "label": "Si",
      "properties": {}
    },
    {
      "species": [
        {
          "element": "Li",
          "occu": 1
        }
      ],
      "abc": [0.5, 0.5, 0.5],
      "xyz": [0.5, 0.5, 0.5],
      "label": "Li",
      "properties": {}
    }
  ],
  "lattice": {
    "matrix": [
      [3.866002, 0, 2.243049],
      [3.866002, 3.866002, 0],
      [0, 3.866002, 2.243049]
    ],
    "a": 5.431,
    "b": 5.431,
    "c": 5.431,
    "alpha": 109.471219,
    "beta": 109.471219,
    "gamma": 109.471219,
    "volume": 129.584354898
  },
  "properties": {
    "space_group": {
      "number": 227,
      "source": "spglib",
      "symbol": "Fd-3m"
    }
  }
}
)

structure.to(filename="../experiments/05-crystal-Si-Li-interstitial-gpt3.5.cif", fmt="cif")
structure.to_ase_atoms().write("../experiments/05-crystal-Si-Li-interstitial-gpt3.5.xyz")